# local_welfare 전처리정리

In [9]:
import pandas as pd
import re

# CSV 불러오기
file_path = "local_final_welfare_list.csv"
df = pd.read_csv(file_path)

# ==========================
# 1. 카테고리 컬럼 불필요한 값 제거
# ==========================
remove_list = [
    "화면크기", "님", "로그아웃까지 남은 시간", "30:00",
    "대한민국 국민 누구나!", "지자체 복지서비스", "찜하기",
    "최종 수정일(반영일)", "지원주기", "제공유형", "처리절차"
]

def clean_category(val):
    if pd.isna(val):
        return val
    items = [x.strip() for x in str(val).split(",")]
    filtered = []
    for x in items:
        if len(x) == 10 and x[4] == "-" and x[7] == "-":
            continue
        if "회성" in x:
            continue
        if any(r == x or r in x for r in remove_list):
            continue
        filtered.append(x)
    return ", ".join(filtered)

df["카테고리"] = df["카테고리"].apply(clean_category)

# ==========================
# 2. 신청방법_x → 신청환경
# ==========================
if "신청방법_x" in df.columns:
    df.rename(columns={"신청방법_x": "신청환경"}, inplace=True)

    def normalize_env(val):
        if pd.isna(val):
            return val
        val = str(val).replace(" ", "")
        if "방문" in val and "인터넷" in val:
            return "ALL"
        elif "방문" in val:
            return "방문"
        elif "인터넷" in val:
            return "인터넷"
        return val

    df["신청환경"] = df["신청환경"].apply(normalize_env)

# ==========================
# 3. 신청방법_y → 신청방법상세
# ==========================
if "신청방법_y" in df.columns:
    df.rename(columns={"신청방법_y": "신청방법상세"}, inplace=True)

    def clean_detail(val):
        if pd.isna(val):
            return val
        text = str(val).strip()
        # 맨 앞 "-" 제거
        text = re.sub(r"^-\s*", "", text)
        # 나머지 "-" 를 "," 로 변환
        text = text.replace("-", ",")
        return text.strip()

    df= df.applymap(lambda x: clean_detail(x))

# ==========================
# 저장
# ==========================
output_path = "local_final_welfare_list_cleaned_copied.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 전처리 완료! 저장된 파일:", output_path)


C:\Users\Sejin\AppData\Local\Temp\ipykernel_8732\2716900351.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df= df.applymap(lambda x: clean_detail(x))


✅ 전처리 완료! 저장된 파일: local_final_welfare_list_cleaned_copied.csv


In [11]:
import pandas as pd

# CSV 불러오기
file_path = "local_final_welfare_list_cleaned_copied.csv"
df = pd.read_csv(file_path)

# 카테고리 사전 정의 (키워드 매핑)
category_keywords = {
    "생계지원": ["식비", "주거비", "공과금", "차상위", "생활비", "긴급복지", "할인"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "보호자", "양육", "간호"],
    "건강·의료": ["질병", "진단", "치료", "의료비", "병원", "임신", "출산"],
    "일상생활": ["가사", "식사", "이동", "위생", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "안전"],
    "주거지원": ["거처", "임대", "전세", "월세", "보증금", "주거환경"],
    "일자리": ["구직", "자활", "취업", "직업", "훈련", "능력개발"],
    "아동지원": ["아동", "보육", "교육", "양육상담", "돌봄"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚"],
    "고립·고독": ["고독", "고립", "사회적", "외로움", "고독사"]
}

# 카테고리 우선순위 정의
priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

# 카테고리 분류 함수 (항상 하나 선택)
def assign_single_category(row):
    text = ""
    for col in ["서비스 요약", "서비스내용", "신청방법상세"]:
        if col in df.columns and pd.notna(row[col]):
            text += " " + str(row[col])
    matched = [cat for cat, keywords in category_keywords.items() if any(k in text for k in keywords)]
    for cat in priority_order:  # 우선순위대로 하나만 선택
        if cat in matched:
            return cat
    return "생계지원"  # 매칭 없으면 기본값

# 카테고리 갱신
df["카테고리"] = df.apply(assign_single_category, axis=1)

# 저장
output_path = "local_final_welfare_list_categorized.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 카테고리 분류 완료, 저장된 파일:", output_path)


✅ 카테고리 분류 완료, 저장된 파일: local_final_welfare_list_categorized.csv


# 중앙부 전처리

In [12]:
import pandas as pd
import re

# 파일 불러오기
file_path = "goverment_final_welfare_list_with_features.csv"
df = pd.read_csv(file_path)

# 1. 상세링크 추가
df["상세링크"] = df["서비스ID"].apply(
    lambda x: f"https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={x}&wlfareInfoReldBztpCd=01"
)

# 2. 대상특성 추출 함수
def extract_target_feature(text):
    if not text:
        return ""
    text = str(text)

    # 나이 범위
    match_range = re.search(r"(\d{1,2})\s*~\s*(\d{1,2})세", text)
    if match_range:
        return f"{match_range.group(1)}~{match_range.group(2)}세"
    match_over = re.search(r"만?\s*(\d{1,2})세\s*이상", text)
    if match_over:
        return f"{match_over.group(1)}세 이상"
    match_under = re.search(r"만?\s*(\d{1,2})세\s*이하", text)
    if match_under:
        return f"{match_under.group(1)}세 이하"

    # 키워드 매핑
    if re.search(r"(청년|청소년)", text):
        return "청년"
    if re.search(r"(노인|노년|고령|어르신)", text):
        return "노년"
    if re.search(r"(저소득|차상위|기초생활|중위소득)", text):
        return "저소득"
    if re.search(r"(임신|출산|산모|임부)", text):
        return "임산부"

    return ""

# 지원대상상세 기반 대상특성 생성
if "지원대상상세" in df.columns:
    df["대상특성"] = df["지원대상상세"].apply(extract_target_feature)
else:
    df["대상특성"] = ""

# 3. 카테고리 분류
category_keywords = {
    "생계지원": ["식비", "주거비", "공과금", "생활비", "차상위", "긴급복지", "할인"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "보호자", "양육", "간호"],
    "건강·의료": ["질병", "진단", "치료", "의료비", "병원", "임신", "출산"],
    "일상생활": ["가사", "식사", "이동", "위생", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "안전"],
    "주거지원": ["거처", "임대", "전세", "월세", "보증금", "주거환경"],
    "일자리": ["구직", "자활", "취업", "직업", "훈련", "능력개발"],
    "아동지원": ["아동", "보육", "교육", "양육상담", "돌봄"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚"],
    "고립·고독": ["고독", "고립", "사회적", "외로움", "고독사"]
}

priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

def assign_category(text):
    if not text:
        return "생계지원"  # 기본값
    text = str(text)
    matched = [cat for cat, keywords in category_keywords.items() if any(k in text for k in keywords)]
    for cat in priority_order:
        if cat in matched:
            return cat
    return "생계지원"

# 서비스개요와 서비스명 기준으로 카테고리 분류
if "서비스개요" in df.columns:
    df["카테고리"] = df["서비스개요"].apply(assign_category)
else:
    df["카테고리"] = df["서비스명"].apply(assign_category)

# 4. 저장
output_path = "goverment_final_welfare_list_categorized.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ CSV 저장 완료:", output_path)


✅ CSV 저장 완료: goverment_final_welfare_list_categorized.csv


In [13]:
import pandas as pd
import os

# 원본 파일 경로
input_path = "private_final_welfare_list.csv"

# 출력 파일 경로
base, ext = os.path.splitext(input_path)
output_path = base + "_no_empty_cols" + ext

# 데이터 불러오기
df = pd.read_csv(input_path, dtype=object)  # 모든 칼럼을 object로 읽으면 공백처리 안전

def col_has_any_value(s: pd.Series) -> bool:
    """
    시리즈가 '모두 빈 값'인지 검사.
    - NaN은 빈값으로 처리
    - 문자열일 경우 공백만 있는 것도 빈값으로 처리
    - 숫자/기타 non-NaN 값은 값이 있는 것으로 간주
    """
    # NaN을 빈 문자열로 대체(벡터화)
    filled = s.where(s.notna(), "")
    # 문자열로 변환 후 양쪽 공백 제거
    stripped = filled.astype(str).str.strip()
    # 하나라도 빈문자열이 아닌 값이 있으면 True
    return (stripped != "").any()

# 각 컬럼에 대해 비어있는지 검사
has_value_mask = df.apply(col_has_any_value, axis=0)
kept_columns = has_value_mask[has_value_mask].index.tolist()
dropped_columns = has_value_mask[~has_value_mask].index.tolist()

print("유지할 컬럼 개수:", len(kept_columns))
print("제거할 빈 컬럼들 (총 {}개):".format(len(dropped_columns)))
for c in dropped_columns:
    print(" -", c)

# 빈 컬럼 제거한 데이터프레임
df_clean = df.loc[:, kept_columns]

# 저장
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")
print("저장 완료:", output_path)


유지할 컬럼 개수: 18
제거할 빈 컬럼들 (총 8개):
 - 대표연락처
 - 신청가능여부
 - 주소
 - 태그
 - 사업상태
 - 사업기간
 - 연락처
 - 이메일
저장 완료: private_final_welfare_list_no_empty_cols.csv


In [16]:
import pandas as pd
import re
import os

input_path = "private_final_welfare_list.csv"
output_path = "private_final_welfare_list_standardized.csv"

# 안전하게 문자열로 읽기
df = pd.read_csv(input_path, dtype=str)

# (생략: 연령 추출 함수, 컬럼 rename 등 기존 코드 유지)
# --- 여기에 앞서 사용하신 derive_age_from_text, 나이 생성 코드 등을 넣으시면 됩니다 ---
# (예: derive_age_from_text 정의, '나이(코드기반)'/'나이' 생성 등)

# ----------------------------
# 빈 칼럼 검사 함수 (안전한 버전)
# ----------------------------
def col_has_any_value(s):
    """
    시리즈나 1-컬럼 DataFrame을 받아서 '하나라도 값이 있는지' 검사.
    - NaN/None -> 빈값
    - 공백 문자열 -> 빈값
    - 리스트/숫자/기타 객체도 str()로 변환 후 공백 판정
    """
    # 만약 DataFrame이 들어오면 모든 원소를 순회하도록 flatten
    if isinstance(s, pd.DataFrame):
        iterator = s.itertuples(index=False, name=None)
        for row in iterator:
            # row는 튜플일 수 있음: 각 원소 검사
            for val in row:
                if val is None:
                    continue
                v = str(val).strip()
                if v != "":
                    return True
        return False
    # Series인 경우 (일반적)
    for val in s:
        if val is None or (isinstance(val, float) and pd.isna(val)):
            continue
        if str(val).strip() != "":
            return True
    return False

# ----------------------------
# 예: 기존 rename_map 적용 (사용자 코드 그대로 유지)
# ----------------------------
rename_map = {
    "서비스ID": "서비스ID",
    "서비스명": "서비스명",
    "서비스 요약": "서비스개요",
    "서비스개요": "서비스개요",
    "상세 링크": "상세링크",
    "상세링크": "상세링크",
    "소관기관": "소관부처",
    "대표연락처": "대표연락처",
    "시작일": "시작일",
    "종료일": "종료일",
    "진행상태": "사업상태",
    "INTRS_THEMA_CD": "관심주제코드",
    "FMLY_CIRC_CD": "대상특성코드",
    "BKJR_LFTM_CYC_CD": "생애주기코드",
    "WLFARE_INFO_AGGRP_CD": "나이",
    "카테고리": "카테고리",
    "사업목적": "서비스목적",
    "지원대상": "지원대상상세",
    "지원대상상세": "지원대상상세",
    "지원내용": "서비스내용",
    "서비스내용": "서비스내용",
    "신청방법": "신청방법상세",
    "신청방법상세": "신청방법상세",
    "제출서류": "필요서류",
}

existing_map = {k:v for k,v in rename_map.items() if k in df.columns}
df = df.rename(columns=existing_map)

# ----------------------------
# 빈 칼럼 감지 및 삭제 (안전)
# ----------------------------
empty_cols = [c for c in df.columns if not col_has_any_value(df[c])]
if empty_cols:
    print("삭제할 빈 컬럼들:", empty_cols)
    df = df.drop(columns=empty_cols)
else:
    print("빈 컬럼 없음. 삭제할 항목 없음.")

# ----------------------------
# 저장
# ----------------------------
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("저장 완료:", os.path.abspath(output_path))


삭제할 빈 컬럼들: ['대표연락처', '신청가능여부', '주소', '태그', '사업기간', '연락처', '이메일']
저장 완료: c:\Users\Sejin\OneDrive\바탕 화면\seoul_ai_hackerton\code\persona-signal-welfare\src\modules\welfare_recommender\crawlers\persona-signal-welfare\src\modules\welfare_recommender\crawlers\datapotal\private_final_welfare_list_standardized.csv


In [17]:
import pandas as pd
import os
import re

# 입력 / 출력 파일 경로 (필요하면 경로를 수정)
INPUT_PATH = "private_final_welfare_list.csv"
OUTPUT_PATH = "private_final_welfare_list_with_category.csv"

# 1) 카테고리 키워드 사전 (체크리스트 기준)
category_keywords = {
    "생계지원": ["생계", "식비", "주거비", "공과금", "생활비", "긴급복지", "할인", "생활지원", "바우처"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "돌봄", "보호", "양육", "방문간호", "돌봄서비스"],
    "건강·의료": ["질병", "진단", "치료", "의료", "병원", "건강검진", "의료비", "임신", "출산", "진료"],
    "일상생활": ["가사", "식사", "이동", "위생", "일상생활", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "재난", "안전", "위험"],
    "주거지원": ["거처", "주거", "전세", "월세", "보증금", "주거환경", "주택", "임대", "주택수리"],
    "일자리": ["구직", "자활", "취업", "직업", "일자리", "직업능력", "훈련", "직업훈련"],
    "아동지원": ["아동", "보육", "양육", "유아", "영유아", "초등", "청소년"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚", "채권", "부채"],
    "고립·고독": ["고립", "고독", "외로움", "사회적고립", "고독사"]
}

# 2) 우선순위 (중복 매칭 시 이 순서로 먼저 선택)
priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

# 3) 파일 읽기
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"입력 파일이 없습니다: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH, dtype=str)

# 4) 자동으로 검색할 텍스트 칼럼 후보 (존재하면 사용)
text_candidates = [
    "서비스개요", "서비스 요약", "서비스내용", "지원내용",
    "지원대상", "지원대상상세", "선정기준", "신청방법상세", "서비스명"
]
text_cols = [c for c in text_candidates if c in df.columns]

# 5) lower-case 키워드 준비
category_keywords_lower = {k: [kw.lower() for kw in kws] for k,kws in category_keywords.items()}

# 6) 행 텍스트 결합 함수
def combined_text_for_row(row):
    parts = []
    for c in text_cols:
        v = row.get(c, "")
        if pd.notna(v) and str(v).strip() != "":
            parts.append(str(v))
    return " ".join(parts).lower()  # 소문자로 변환하여 매칭 단순화

# 7) 카테고리 할당 함수
def assign_category(row):
    text = combined_text_for_row(row)
    matched = []
    for cat, kws in category_keywords_lower.items():
        for kw in kws:
            if kw in text:
                matched.append(cat)
                break
    # 우선순위에 따라 하나만 선택
    for cat in priority_order:
        if cat in matched:
            return cat
    # 아무것도 매칭되지 않으면 기본값 '생계지원'
    return "생계지원"

# 8) 카테고리 컬럼 생성 (덮어쓰기)
df["카테고리"] = df.apply(assign_category, axis=1)

# 9) 결과 간단 통계 출력
print("카테고리 분포:")
print(df["카테고리"].value_counts(dropna=False))

# 10) 저장
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print("저장 완료:", OUTPUT_PATH)


카테고리 분포:
카테고리
생계지원     136
아동지원      90
건강·의료     39
일자리       27
주거지원      15
채무·법률     11
돌봄·보호     10
안전위기       8
Name: count, dtype: int64
저장 완료: private_final_welfare_list_with_category.csv
